In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# Clone repo

In [2]:
!git clone https://github.com/kaylode/vnm-ocr-toolbox.git main
%cd main

Cloning into 'main'...
remote: Enumerating objects: 1246, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1246 (delta 54), reused 58 (delta 37), pack-reused 1166 (from 1)
Receiving objects: 100% (1246/1246), 23.63 MiB | 29.02 MiB/s, done.
Resolving deltas: 100% (622/622), done.
/content/main


In [3]:
%cd main
!git checkout master
!git reset --hard HEAD
!git pull

[Errno 2] No such file or directory: 'main'
/content/main
Already on 'master'
Your branch is up to date with 'origin/master'.
HEAD is now at ef8744b update dataset link
Already up to date.


# Install dependencies

In [4]:
%%capture
%cd /content/main/
!pip install -r requirements.txt

# Download pretrained weights

In [5]:
!pip install pylsd-nova # Install pylsd-nova instead of pylsd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylsd-nova: filename=pylsd_nova-1.2.1-py3-none-any.whl size=172444 sha256=174790c5c683b8d81a384c70fe4d4af0d8dcdc24de6745e1f0999164f8b22947
  Stored in directory: /root/.cache/pip/wheels/89/73/ea/c0aafa24ff079fe0ea8cf7382fd4e7c5aa5d22989046eb827a
Successfully built pylsd-nova


In [6]:
!pip install shapely # Cài đặt shapely cho đối tượng Polygon

In [7]:
!pip install pyclipper

  Using cached pyclipper-1.3.0.post6-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (9.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 33.3 MB/s eta 0:00:00


In [8]:
# ipython-input-15-e8ebebf70510
%cd /content/main
# !mkdir "/content/main/weights"

# Di chuyển lệnh import download_pretrained_weights đến đây
from tool.utils import download_pretrained_weights

# Bây giờ gọi download_pretrained_weights
download_pretrained_weights("pan_resnet18_sroie19", cached="/content/main/weights/PANNet_best_map.pth")
download_pretrained_weights("transformerocr_mcocr", cached="/content/main/weights/transformerocr.pth")
download_pretrained_weights("phobert_mcocr", cached="/content/main/weights/phobert_report.pth")

/content/main


ModuleNotFoundError: No module named 'torchvision.models.utils'

# Inference

## Inference script

In [ ]:
%cd /content/main
from IPython.display import Image
from IPython.display import display



!python run.py --input=/content/image.png --output=/content/main/results --do_retrieve --debug --find_best_rotation

img1 = Image(f"/content/main/data/mcocr/images/val/{IMAGE_ID}.jpg", width=600)
img2 = Image(f"/content/main/results/{IMAGE_ID}/cache/preprocessed.jpg", width=600)
img3 = Image(f"/content/main/results/{IMAGE_ID}/cache/detected.jpg", width=600)
img4 = Image(f"/content/main/results/{IMAGE_ID}/result.jpg", width=600)

display(img1,img2,img3,img4)

In [ ]:
%cd /content/main
from IPython.display import Image
from IPython.display import display

IMAGE_ID = "201be0a2fea60af853b7"

!python run.py --input=/content/"{IMAGE_ID}".jpg --output=/content/main/results/"{IMAGE_ID}" --do_retrieve --debug --find_best_rotation

img1 = Image(f"/content/main/data/mcocr/images/val/{IMAGE_ID}.jpg", width=600)
img2 = Image(f"/content/main/results/{IMAGE_ID}/cache/preprocessed.jpg", width=600)
img3 = Image(f"/content/main/results/{IMAGE_ID}/cache/detected.jpg", width=600)
img4 = Image(f"/content/main/results/{IMAGE_ID}/result.jpg", width=600)

display(img1,img2,img3,img4)

## Inference modules on Personal ID

In [ ]:
%cd /content/main
import os
import cv2
import re
import pandas as pd
from modules import Preprocess, Detection, OCR, Correction
from tool.utils import natural_keys, visualize
import time
import matplotlib.pyplot as plt

In [ ]:
# Define some variables
img_id = "cmnd_Ngc_M_1_0012"

det_weight = None
ocr_weight = None

In [ ]:
# Read image
img = cv2.imread(f"/content/drive/MyDrive/AI Competitions/IDCard parser/aug/{img_id}.jpg")

plt.imshow(img)
plt.show()

In [ ]:
# Initialize modules

det_model = Detection(weight_path=det_weight)
ocr_model = OCR(weight_path=ocr_weight)
preproc = Preprocess(
    det_model=det_model,
    ocr_model=ocr_model,
    find_best_rotation=False)
correction = Correction()

In [ ]:
# Preprocess image

img1 = preproc(img)

plt.imshow(img1)
plt.show()

In [ ]:
# Detect texts

boxes, img2  = det_model(
    img1,
    crop_region=True,                               #Crop detected regions for OCR
    return_result=True,                             # Return plotted result
    output_path=f"/content/main/results/{img_id}"   #Path to save cropped regions
)

plt.imshow(img2)
plt.show()

In [ ]:
# Text OCR

img_paths=os.listdir(f"/content/main/results/{img_id}/crops") # Cropped regions
img_paths.sort(key=natural_keys)
img_paths = [os.path.join(f"/content/main/results/{img_id}/crops", i) for i in img_paths]

texts, probs = ocr_model.predict_folder(img_paths, return_probs=True) # OCR
texts = correction(texts)   # Word correction

for i in texts:
    print(i)

In [ ]:
preds, probs = None, None
visualize(
    img1, boxes, texts,
    img_name = f'/content/main/results/{img_id}/result.jpg',
    class_mapping=class_mapping,
    labels = preds, probs = probs,
    visualize_best=False)


from IPython.display import Image
from IPython.display import display

img = Image(f"/content/main/results/{img_id}/result.jpg", width=600)
display(img)

In [ ]:
preds, probs = None, None
visualize(
    img1, boxes, texts,
    img_name = f'/content/main/results/{img_id}/result.jpg',
    class_mapping=class_mapping,
    labels = preds, probs = probs,
    visualize_best=False)

end = time.time()
print(f"Executed in {end-start} s")

In [ ]:
!rm -rf ".cache"

## Inference modules on Invoice

In [ ]:
%cd /content/main
import os
import cv2
import re
import pandas as pd
from modules import Preprocess, Detection, OCR, Retrieval, Correction
from tool.utils import natural_keys, visualize
import time
import matplotlib.pyplot as plt

In [ ]:
# Define some auxiliary functions

def find_highest_score_each_class(labels, probs):
        best_score = [0] * (len(class_mapping.keys())-1)
        best_idx = [-1] * (len(class_mapping.keys())-1)
        for i, (label, prob) in enumerate(zip(labels, probs)):
            label_idx = class_mapping[label]
            if label_idx != class_mapping["NONE"]:
                if prob > best_score[label_idx]:
                    best_score[label_idx] = prob
                    best_idx[label_idx] = i
        return best_idx

def find_total_cost_value(total_cost_idx, boxes):
    total_cost_box = boxes[total_cost_idx]
    x1,y1 = total_cost_box[0]
    for i in range(total_cost_idx+1, len(boxes)):
        x1_,y1_ = boxes[i][0]

        if abs(x1-x1_) < 2:
          return i-1


def extract_timestamp(text):
    x = re.findall(r'\d{2}:\d{2}|\d{2}:\d{2}:\d{2}|\d{2}-\d{2}-\d{2}|\d{2}\.\d{2}\.\d{2}|\d+/\d+/\d+', text)
    return ' '.join(x)

In [ ]:
# Define some variables
img_id = "mcocr_val_145115bxsnt"
class_mapping = {"SELLER":0, "ADDRESS":1, "TIMESTAMP":2, "TOTAL_COST":3, "NONE":4}
idx_mapping = {0:"SELLER", 1:"ADDRESS", 2:"TIMESTAMP", 3:"TOTAL_COST", 4:"NONE"}

det_weight = "/content/drive/MyDrive/AI Competitions/MC-OCR/checkpoints/detection-checkpoints/best_rotation/PANNet_best_map.pth"
ocr_weight = "/content/drive/MyDrive/AI Competitions/MC-OCR/checkpoints/ocr-checkpoints/transformerocr.pth"

In [ ]:
# Read image
img = cv2.imread(f"/content/main/data/mcocr/images/val/{img_id}.jpg")

plt.imshow(img)
plt.show()

In [ ]:
# Initialize modules

det_model = Detection(weight_path=det_weight)
ocr_model = OCR(weight_path=ocr_weight)
preproc = Preprocess(
    det_model=det_model,
    ocr_model=ocr_model,
    find_best_rotation=False)
retrieval = Retrieval(class_mapping, mode = 'all')
correction = Correction()

In [ ]:
# Preprocess image

img1 = preproc(img)

plt.imshow(img1)
plt.show()

In [ ]:
# Detect texts

boxes, img2  = det_model(
    img1,
    crop_region=True,                               #Crop detected regions for OCR
    return_result=True,                             # Return plotted result
    output_path=f"/content/main/results/{img_id}"   #Path to save cropped regions
)

plt.imshow(img2)
plt.show()

In [ ]:
# Text OCR

img_paths=os.listdir(f"/content/main/results/{img_id}/crops") # Cropped regions
img_paths.sort(key=natural_keys)
img_paths = [os.path.join(f"/content/main/results/{img_id}/crops", i) for i in img_paths]

texts, probs = ocr_model.predict_folder(img_paths, return_probs=True) # OCR
texts = correction(texts)   # Word correction

for i in texts:
    print(i)

In [ ]:
preds, probs = retrieval(texts)

# Custom post-process
best_score_idx = find_highest_score_each_class(preds, probs)
value_idx = find_total_cost_value(best_score_idx[3], boxes)
if value_idx != best_score_idx[3]:
    texts[best_score_idx[3]] += (" " + texts[value_idx])

texts[best_score_idx[2]] = extract_timestamp(texts[best_score_idx[2]])

# Visualize and save result
visualize(
    img1, boxes, texts,
    img_name = f'/content/main/results/{img_id}/result.jpg',
    class_mapping=class_mapping,
    labels = preds, probs = probs,
    visualize_best=True)

with open(f"/content/main/results/{img_id}/result.txt", 'w') as f:
    for cls, idx in enumerate(best_score_idx):
        f.write(f"{idx_mapping[cls]} : {texts[idx]}\n")

from IPython.display import Image
from IPython.display import display

img = Image(f"/content/main/results/{img_id}/result.jpg", width=600)
display(img)